In [1]:
import DMN3 as DMN
from matplotlib import pyplot as plt
import numpy as np
from helperfuncs import *

In [2]:
node = DMN.Node(np.ones(6)*2)
node.compliance
node.rotated_compliance = node.compliance

node.left = DMN.Node(np.ones(6)*9)
node.right = DMN.Node(np.ones(6)*3)

node.right.rotated_compliance = node.right.compliance
node.left.rotated_compliance = node.left.compliance


In [3]:
node.sigma = np.ones(3)*2
DMN.update_stress(node)


In [4]:
node.right.sigma

array([1.33333333, 2.        , 2.        ])

In [5]:
node.left.sigma

array([0.44444444, 2.        , 2.        ])